# Web scraping for most of the sitemap websites

In [3]:
# importing libraries

import csv 
import operator
import re
import threading

import sys

import requests
from tqdm import tqdm 
from bs4 import BeautifulSoup

from concurrent.futures import ThreadPoolExecutor
import concurrent

from urllib.parse import urlparse
from urllib.parse import urljoin

import spacy # we use this for word similarity

from collections import defaultdict
from fake_useragent import UserAgent

ua = UserAgent()

### Loading the links for scraping

In [4]:
links = []

with open('../link_data.csv', mode='r', newline='', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        links.append(row[0])

### Methods that will be used for scraping


In [ ]:
def get_data(url, retries=3):
    headers = {
        'User-Agent': ua.random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        if retries > 0:
            time.sleep(2 ** (4 - retries))  # Exponential backoff
            return get_data(url, retries - 1)
        return None